In [1]:
import psycopg2
import numpy as np
import json
from gensim import corpora, models, similarities
from gensim.models.doc2vec import LabeledSentence
import nltk
import re

<h1> connect to postgres db and pull quesiton and answer text</h1>

In [2]:
conn = psycopg2.connect("host=localhost port=5432 dbname=amazon user=postgres password=darkmatter")
cur = conn.cursor()

In [3]:
cur.execute("SELECT question from qa;")
Qresults=cur.fetchall()
cur.execute("SELECT answer from qa;")
Aresults=cur.fetchall()

<h1> clean up each question/answer and tokenize </h1>

In [4]:
def process_line(sentence):
    #step 1 split if we need to
    sentences=re.split(r"(?<![0-9])[.?!;](?![0-9])", sentence)
    result= [re.findall("[a-z'.-0-9]+", sent.lower()) for sent in sentences if \
            re.findall("[a-z'.-0-9]+", sent.lower())!=[]]
    if result==[]:
        result=[['']]
    return result
stoplist = set('for a of the and to in rt'.split())

In [5]:
#sentence ="this doesn't work so well ?   why"
#filter_text=' '.join(re.findall("[a-z'?]+", sentence.lower()))
#filter_text.replace('?',' ? ').split()
#nltk.word_tokenize(filter_text)

<h1> define stop words and apply </h1>

In [6]:
stoplist = set('for of the and to in rt'.split())
stoplist

{'and', 'for', 'in', 'of', 'rt', 'the', 'to'}

In [7]:
question_sentence= [[word for word in sum(process_line(sentence[0]),[]) if word not in stoplist] for sentence in Qresults]

In [103]:
question_sentence[9]

['is',
 'this',
 'badger',
 'compatible',
 'with',
 'badger',
 'model',
 '?',
 'same',
 'size',
 'fittings',
 '?']

<h1> train word2Vec model </h1>

In [104]:
Qmodel = models.Word2Vec(question_sentence, size=100, window=5, min_count=10, workers=8)

In [55]:
Qmodel.most_similar('')

[('or', 0.6198657751083374),
 ('also', 0.6104353070259094),
 ('ie', 0.47483423352241516),
 ('it', 0.44304147362709045),
 ('this', 0.43430107831954956),
 ('that', 0.4327320456504822),
 ('meaning', 0.42874017357826233),
 ('does', 0.4074341654777527),
 ('thanks', 0.4056839942932129),
 ('with', 0.4022674560546875)]

<h1> train biagram </h1>

In [8]:
Qbigram = models.Phrases(question_sentence,min_count=5) #model bigrams
Qbigrams=list(Qbigram[question_sentence]) #make list
QmodelB = models.Word2Vec(Qbigrams, size=100, window=5, min_count=5, workers=8) #train

In [9]:
QmodelB.most_similar('does') #positive=['finna', 'gonna'], negative=['bro'],topn=20)

[('dose', 0.8526506423950195),
 ('doe', 0.7331932783126831),
 ("doesn't", 0.6430281400680542),
 ('will', 0.6202369928359985),
 ('dos', 0.6173581480979919),
 ('doesnt', 0.5512901544570923),
 ('did', 0.5015972852706909),
 ("doesn_'t", 0.4602945148944855),
 ('ability', 0.4364972710609436),
 ("won't", 0.4283829927444458)]

In [10]:
QmodelB.save('/home/ubuntu/TallLabs/models/QmodelB')

<h1> Train Doc2vec </h1>

In [57]:
class lb_sentence:
    def __init__(self, sentence_list,tag):
        self.words=sentence_list
        self.tags=tag
    

In [62]:
#lb_qs=LabeledSentence(question_sentence,1)
lb_qs=[lb_sentence(sen ,np.ones(len(sen)))for sen in question_sentence]

In [63]:
model = models.Doc2Vec(lb_qs, size=100, window=8, min_count=5, workers=4)

In [ ]:
model =  models.Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
model.build_vocab(lb_qs)
for epoch in range(10):
    model.train(lb_qs)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

In [69]:
model.most_similar('charger')

[('recharger', 0.7499253153800964),
 ('chargeable', 0.632102906703949),
 ('rechargeable', 0.6074789762496948),
 ('lithium', 0.5788451433181763),
 ('chargable', 0.5748066902160645),
 ('lithiom', 0.5646908283233643),
 ('rechargable', 0.5600529313087463),
 ('gpa', 0.5514222383499146),
 ('toothbrush', 0.5457131266593933),
 ('nicad', 0.5350093245506287)]

In [71]:
import lda2vec